# Pivot Tables & Reshaping

Vamos a ver algunas técnicas para manejar DataFrames

In [53]:
import pandas as pd
import numpy as np

In [54]:
import pandas.util.testing as tm; tm.N = 3
def unpivot(frame):
    N, K = frame.shape
    data = {'value' : frame.values.ravel('F'),
            'variable' : np.asarray(frame.columns).repeat(N),
            'date' : np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])
df = unpivot(tm.makeTimeDataFrame())

In [55]:
df.head(10)

,date,variable,value
0,2000-01-03,A,-0.918807
1,2000-01-04,A,-0.446067
2,2000-01-05,A,-1.913497
3,2000-01-03,B,0.001977
4,2000-01-04,B,-0.477170
5,2000-01-05,B,0.525968
6,2000-01-03,C,0.332468
7,2000-01-04,C,-0.047402
8,2000-01-05,C,0.358489
9,2000-01-03,D,-1.054124


In [56]:
df.pivot(index='date',columns='variable',values='value')

variable,A,B,C,D
date,,,,
2000-01-03,-0.918807,0.001977,0.332468,-1.054124
2000-01-04,-0.446067,-0.477170,-0.047402,-0.022943
2000-01-05,-1.913497,0.525968,0.358489,-1.221067


Si tenemos más de un valor y omitimos pasar el parámetro 'values', entonces pandas creara una serie de columnas jerárquicas por nosotros. Ejemplo:

Creamos una columna adicional 'value2'

In [57]:
df['value2']= df['value']*2

In [58]:
df.head()

,date,variable,value,value2
0,2000-01-03,A,-0.918807,-1.837613
1,2000-01-04,A,-0.446067,-0.892134
2,2000-01-05,A,-1.913497,-3.826993
3,2000-01-03,B,0.001977,0.003955
4,2000-01-04,B,-0.477170,-0.954340


In [59]:
pivoted = df.pivot('date','variable')
pivoted

value                                  value2            \
variable           A         B         C         D         A         B   
date                                                                     
2000-01-03 -0.918807  0.001977  0.332468 -1.054124 -1.837613  0.003955   
2000-01-04 -0.446067 -0.477170 -0.047402 -0.022943 -0.892134 -0.954340   
2000-01-05 -1.913497  0.525968  0.358489 -1.221067 -3.826993  1.051936   

                                
variable           C         D  
date                            
2000-01-03  0.664937 -2.108247  
2000-01-04 -0.094804 -0.045887  
2000-01-05  0.716979 -2.442134

In [60]:
type(pivoted)

pandas.core.frame.DataFrame

A la que podemos accesar escribiendo por ejemplo:

In [61]:
pivoted['value2']

variable,A,B,C,D
date,,,,
2000-01-03,-1.837613,0.003955,0.664937,-2.108247
2000-01-04,-0.892134,-0.954340,-0.094804,-0.045887
2000-01-05,-3.826993,1.051936,0.716979,-2.442134


## Reshaping by stacking and unstacking

In [62]:
tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                         'foo', 'foo', 'qux', 'qux'],
                        ['one', 'two', 'one', 'two',
                         'one', 'two', 'one', 'two']]))

In [63]:
?zip

In [64]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [65]:
index = pd.MultiIndex.from_tuples(tuples,names=['first','second'])
index

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [66]:
df2 = pd.DataFrame(np.random.randn(8,2),index=index, columns=['A','B'])
df2

A         B
first second                    
bar   one    -0.389612 -0.776616
      two    -2.618566 -0.962490
baz   one    -0.873868 -1.282700
      two    -1.840736  1.238781
foo   one    -0.713971  0.655316
      two    -0.721985  0.876769
qux   one     1.547715 -0.226763
      two    -0.388761 -0.229712

La función **stack** comprime un nivel las columnas del DataFrame para producir ya sea:
<li> Una Serie, en el caso de un índice de una sóla columna </li>
<li> Un DataFrame, en el caso de columnas con múltiples índices </li>

In [67]:
stacked = df2.stack()
stacked

first  second   
bar    one     A   -0.389612
               B   -0.776616
       two     A   -2.618566
               B   -0.962490
baz    one     A   -0.873868
               B   -1.282700
       two     A   -1.840736
               B    1.238781
foo    one     A   -0.713971
               B    0.655316
       two     A   -0.721985
               B    0.876769
qux    one     A    1.547715
               B   -0.226763
       two     A   -0.388761
               B   -0.229712
dtype: float64

La operación inversa de stack es **unstack** que por default descomprime el último nivel

In [68]:
stacked.unstack()

A         B
first second                    
bar   one    -0.389612 -0.776616
      two    -2.618566 -0.962490
baz   one    -0.873868 -1.282700
      two    -1.840736  1.238781
foo   one    -0.713971  0.655316
      two    -0.721985  0.876769
qux   one     1.547715 -0.226763
      two    -0.388761 -0.229712

In [69]:
stacked.unstack?

In [70]:
stacked.unstack().unstack()

A                   B          
second       one       two       one       two
first                                         
bar    -0.389612 -2.618566 -0.776616 -0.962490
baz    -0.873868 -1.840736 -1.282700  1.238781
foo    -0.713971 -0.721985  0.655316  0.876769
qux     1.547715 -0.388761 -0.226763 -0.229712

In [71]:
stacked.unstack(0)

first          bar       baz       foo       qux
second                                          
one    A -0.389612 -0.873868 -0.713971  1.547715
       B -0.776616 -1.282700  0.655316 -0.226763
two    A -2.618566 -1.840736 -0.721985 -0.388761
       B -0.962490  1.238781  0.876769 -0.229712

In [72]:
stacked.unstack(1)

second        one       two
first                      
bar   A -0.389612 -2.618566
      B -0.776616 -0.962490
baz   A -0.873868 -1.840736
      B -1.282700  1.238781
foo   A -0.713971 -0.721985
      B  0.655316  0.876769
qux   A  1.547715 -0.388761
      B -0.226763 -0.229712

Si los niveles tienen nombres como es el caso, entonces podemos referirnos a ellos por nombre

In [73]:
stacked.unstack('second')

second        one       two
first                      
bar   A -0.389612 -2.618566
      B -0.776616 -0.962490
baz   A -0.873868 -1.840736
      B -1.282700  1.238781
foo   A -0.713971 -0.721985
      B  0.655316  0.876769
qux   A  1.547715 -0.388761
      B -0.226763 -0.229712

Nota que los métodos stack y unstack implícitamente ordenan los levels por índice por lo que aplicar unstack y stack nos regresas un nuevo DataFrame ordenado

Generemos un nuevo DataFrame para trabajar y probar nuestra teoría

In [74]:
index = pd.MultiIndex.from_product([[2,1], ['a', 'b']])

In [75]:
df = pd.DataFrame(np.random.randn(4), index=index, columns=['A'])
df

A
2 a -0.675473
  b -0.131259
1 a  0.034070
  b  0.742034

In [76]:
df.sort_index()

A
1 a  0.034070
  b  0.742034
2 a -0.675473
  b -0.131259

In [77]:
df.unstack().stack()

A
1 a  0.034070
  b  0.742034
2 a -0.675473
  b -0.131259

In [78]:
all(df.unstack().stack() == df.sort_index())

True

Comprobada! 

### Trabajando con mútiples levels (niveles)

Primero generemos el DataFrame

In [79]:
columns = pd.MultiIndex.from_tuples([
            ('A', 'cat', 'long'), ('B', 'cat', 'long'),
            ('A', 'dog', 'short'), ('B', 'dog', 'short')
        ],
        names=['exp', 'animal', 'hair_length']
    )

In [80]:
df = pd.DataFrame(np.random.randn(4, 4), columns=columns)
df

exp,A,B,A,B
animal,cat,cat,dog,dog
hair_length,long,long,short,short
0,-2.913886,0.392619,-1.294098,-1.056858
1,-0.808734,-0.627578,1.596067,-1.593356
2,-0.631981,0.790498,0.486945,-0.813341
3,0.126385,-1.554363,0.967148,0.849524


La lista de niveles puede tener sin problema el nombre de 'levels' (los niveles) o el índice de los niveles. Entonces

In [81]:
df.stack(level=['animal', 'hair_length'])

exp                          A         B
  animal hair_length                    
0 cat    long        -2.913886  0.392619
  dog    short       -1.294098 -1.056858
1 cat    long        -0.808734 -0.627578
  dog    short        1.596067 -1.593356
2 cat    long        -0.631981  0.790498
  dog    short        0.486945 -0.813341
3 cat    long         0.126385 -1.554363
  dog    short        0.967148  0.849524

alternativamente

In [82]:
df.stack(level=[1, 2])

exp                          A         B
  animal hair_length                    
0 cat    long        -2.913886  0.392619
  dog    short       -1.294098 -1.056858
1 cat    long        -0.808734 -0.627578
  dog    short        1.596067 -1.593356
2 cat    long        -0.631981  0.790498
  dog    short        0.486945 -0.813341
3 cat    long         0.126385 -1.554363
  dog    short        0.967148  0.849524

### Valores Faltantes (Missing Values)

Generemos primero el DataFrame

In [83]:
columns = pd.MultiIndex.from_tuples([('A', 'cat'), ('B', 'dog'),
                                         ('B', 'cat'), ('A', 'dog')],
                                        names=['exp', 'animal'])
    

In [84]:
index = pd.MultiIndex.from_product([('bar', 'baz', 'foo', 'qux'),
                                        ('one', 'two')],
                                       names=['first', 'second'])
    

In [85]:
df = pd.DataFrame(np.random.randn(8, 4), index=index, columns=columns)
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.066092  0.033665  0.084472 -0.311708
      two    -0.319968  0.088142 -0.082145  0.685613
baz   one     1.651573 -0.387410 -0.851103  0.770073
      two    -0.931478 -0.134764 -0.489235  1.264558
foo   one    -1.302296 -1.967393 -0.730897 -0.113244
      two     1.282136  0.739658  1.168354 -1.153522
qux   one     0.729747  0.598078  0.191942 -1.516747
      two    -0.387333 -0.152005 -0.647101 -1.813038

In [86]:
df2 = df.iloc[[0, 1, 2, 4, 5, 7]]
df2

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.066092  0.033665  0.084472 -0.311708
      two    -0.319968  0.088142 -0.082145  0.685613
baz   one     1.651573 -0.387410 -0.851103  0.770073
foo   one    -1.302296 -1.967393 -0.730897 -0.113244
      two     1.282136  0.739658  1.168354 -1.153522
qux   two    -0.387333 -0.152005 -0.647101 -1.813038

Entonces podemos seleccionar con el método stack para seleccionar a que nivel queremos las columnas a acomodar

In [87]:
df2.stack('exp')

animal                 cat       dog
first second exp                    
bar   one    A    0.066092 -0.311708
             B    0.084472  0.033665
      two    A   -0.319968  0.685613
             B   -0.082145  0.088142
baz   one    A    1.651573  0.770073
             B   -0.851103 -0.387410
foo   one    A   -1.302296 -0.113244
             B   -0.730897 -1.967393
      two    A    1.282136 -1.153522
             B    1.168354  0.739658
qux   two    A   -0.387333 -1.813038
             B   -0.647101 -0.152005

In [88]:
df2.stack('animal')

exp                         A         B
first second animal                    
bar   one    cat     0.066092  0.084472
             dog    -0.311708  0.033665
      two    cat    -0.319968 -0.082145
             dog     0.685613  0.088142
baz   one    cat     1.651573 -0.851103
             dog     0.770073 -0.387410
foo   one    cat    -1.302296 -0.730897
             dog    -0.113244 -1.967393
      two    cat     1.282136  1.168354
             dog    -1.153522  0.739658
qux   two    cat    -0.387333 -0.647101
             dog    -1.813038 -0.152005

Ahora veamos que hacer unstacking puede resultar en valores faltantes si los subgrupos no tienen el mismo conjunto de etiquetas

In [89]:
df3 = df.iloc[[0, 1, 4, 7], [1, 2]]
df3

exp                  B          
animal             dog       cat
first second                    
bar   one     0.033665  0.084472
      two     0.088142 -0.082145
foo   one    -1.967393 -0.730897
qux   two    -0.152005 -0.647101

In [90]:
df3.unstack()

exp            B                              
animal       dog                 cat          
second       one       two       one       two
first                                         
bar     0.033665  0.088142  0.084472 -0.082145
foo    -1.967393       NaN -0.730897       NaN
qux          NaN -0.152005       NaN -0.647101

Para subsanar este problema, unstack toma el argumento 'fill_value' para especificar el tratamiento que le queremos dar a esos valores faltantes (missing values)

In [91]:
df3.unstack(fill_value=-1e9)

exp                B                                          
animal           dog                         cat              
second           one           two           one           two
first                                                         
bar     3.366527e-02  8.814208e-02  8.447242e-02 -8.214478e-02
foo    -1.967393e+00 -1.000000e+09 -7.308974e-01 -1.000000e+09
qux    -1.000000e+09 -1.520052e-01 -1.000000e+09 -6.471005e-01

### Valores faltantes con multi-indexing

In [92]:
df[:3]

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.066092  0.033665  0.084472 -0.311708
      two    -0.319968  0.088142 -0.082145  0.685613
baz   one     1.651573 -0.387410 -0.851103  0.770073

In [93]:
df[:3].unstack(0)

exp            A                   B                                      A  \
animal       cat                 dog                cat                 dog   
first        bar       baz       bar      baz       bar       baz       bar   
second                                                                        
one     0.066092  1.651573  0.033665 -0.38741  0.084472 -0.851103 -0.311708   
two    -0.319968       NaN  0.088142      NaN -0.082145       NaN  0.685613   

exp               
animal            
first        baz  
second            
one     0.770073  
two          NaN

In [94]:
df2.unstack(1)

exp            A                   B                                       A  \
animal       cat                 dog                 cat                 dog   
second       one       two       one       two       one       two       one   
first                                                                          
bar     0.066092 -0.319968  0.033665  0.088142  0.084472 -0.082145 -0.311708   
baz     1.651573       NaN -0.387410       NaN -0.851103       NaN  0.770073   
foo    -1.302296  1.282136 -1.967393  0.739658 -0.730897  1.168354 -0.113244   
qux          NaN -0.387333       NaN -0.152005       NaN -0.647101       NaN   

exp               
animal            
second       two  
first             
bar     0.685613  
baz          NaN  
foo    -1.153522  
qux    -1.813038

### Masajeando un DataFrame con melt

In [95]:
cheese = pd.DataFrame({'first' : ['John', 'Mary'],
                           'last' : ['Doe', 'Bo'],
                           'height' : [5.5, 6.0],
                           'weight' : [130, 150]})
cheese

,first,height,last,weight
0,John,5.5,Doe,130
1,Mary,6.0,Bo,150


In [96]:
cheese.melt(id_vars=['first', 'last'])

,first,last,variable,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [97]:
cheese.melt(id_vars=['first', 'last'], var_name='quantity')

,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


### Combinando estadística y GroupBy

In [98]:
df

exp                  A         B                   A
animal             cat       dog       cat       dog
first second                                        
bar   one     0.066092  0.033665  0.084472 -0.311708
      two    -0.319968  0.088142 -0.082145  0.685613
baz   one     1.651573 -0.387410 -0.851103  0.770073
      two    -0.931478 -0.134764 -0.489235  1.264558
foo   one    -1.302296 -1.967393 -0.730897 -0.113244
      two     1.282136  0.739658  1.168354 -1.153522
qux   one     0.729747  0.598078  0.191942 -1.516747
      two    -0.387333 -0.152005 -0.647101 -1.813038

In [99]:
df.stack()

exp                         A         B
first second animal                    
bar   one    cat     0.066092  0.084472
             dog    -0.311708  0.033665
      two    cat    -0.319968 -0.082145
             dog     0.685613  0.088142
baz   one    cat     1.651573 -0.851103
             dog     0.770073 -0.387410
      two    cat    -0.931478 -0.489235
             dog     1.264558 -0.134764
foo   one    cat    -1.302296 -0.730897
             dog    -0.113244 -1.967393
      two    cat     1.282136  1.168354
             dog    -1.153522  0.739658
qux   one    cat     0.729747  0.191942
             dog    -1.516747  0.598078
      two    cat    -0.387333 -0.647101
             dog    -1.813038 -0.152005

In [100]:
df.stack().mean(1)

first  second  animal
bar    one     cat       0.075282
               dog      -0.139021
       two     cat      -0.201056
               dog       0.386877
baz    one     cat       0.400235
               dog       0.191332
       two     cat      -0.710357
               dog       0.564897
foo    one     cat      -1.016597
               dog      -1.040318
       two     cat       1.225245
               dog      -0.206932
qux    one     cat       0.460845
               dog      -0.459334
       two     cat      -0.517217
               dog      -0.982522
dtype: float64

In [101]:
df.stack().mean(1).unstack()

animal             cat       dog
first second                    
bar   one     0.075282 -0.139021
      two    -0.201056  0.386877
baz   one     0.400235  0.191332
      two    -0.710357  0.564897
foo   one    -1.016597 -1.040318
      two     1.225245 -0.206932
qux   one     0.460845 -0.459334
      two    -0.517217 -0.982522

Alternativamente

In [102]:
df.groupby(level=1, axis=1).mean()

animal             cat       dog
first second                    
bar   one     0.075282 -0.139021
      two    -0.201056  0.386877
baz   one     0.400235  0.191332
      two    -0.710357  0.564897
foo   one    -1.016597 -1.040318
      two     1.225245 -0.206932
qux   one     0.460845 -0.459334
      two    -0.517217 -0.982522

In [43]:
df.stack().groupby(level=1).mean()

exp,A,B
second,,
one,-0.337322,0.319901
two,0.389773,0.002761


In [44]:
df.mean().unstack(0)

exp,A,B
animal,,
cat,-0.258862,0.002925
dog,0.311313,0.319737


### Por fin.... Tablas dinámicas!!

In [103]:
import datetime
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 6,
                       'B': ['A', 'B', 'C'] * 8,
                       'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 4,
                       'D': np.random.randn(24),
                       'E': np.random.randn(24),
                       'F': [datetime.datetime(2013, i, 1) for i in range(1, 13)] +
                            [datetime.datetime(2013, i, 15) for i in range(1, 13)]})
df

,A,B,C,D,E,F
0,one,A,foo,0.091165,-1.091410,2013-01-01
1,one,B,foo,-0.623827,0.367733,2013-02-01
2,two,C,foo,1.744676,0.336085,2013-03-01
3,three,A,bar,0.508173,-1.942918,2013-04-01
4,one,B,bar,0.565398,-1.621291,2013-05-01
5,one,C,bar,-0.081633,-1.077220,2013-06-01
6,two,A,foo,-0.328222,-1.759828,2013-07-01
7,three,B,foo,0.685952,1.276188,2013-08-01
8,one,C,foo,0.776770,1.010452,2013-09-01
9,one,A,bar,-1.044824,-1.490115,2013-10-01


Podemos pedirle que nos haga una tabla dinámica de esta información de la siguiente forma

In [46]:
pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])

C             bar       foo
A     B                    
one   A  0.537120 -1.230993
      B -1.142890  0.917770
      C  0.089520  0.662680
three A  0.941718       NaN
      B       NaN -0.979393
      C  0.948707       NaN
two   A       NaN  0.047028
      B -0.314629       NaN
      C       NaN -0.429362

In [47]:
pd.pivot_table(df, values='D', index=['B'], columns=['A', 'C'], aggfunc=np.sum)

A       one               three                 two          
C       bar       foo       bar       foo       bar       foo
B                                                            
A  1.074239 -2.461986  1.883436       NaN       NaN  0.094057
B -2.285780  1.835541       NaN -1.958787 -0.629257       NaN
C  0.179040  1.325360  1.897414       NaN       NaN -0.858725

In [48]:
pd.pivot_table(df, values=['D','E'], index=['B'], columns=['A', 'C'], aggfunc=np.sum)

D                                                           E  \
A       one               three                 two                 one   
C       bar       foo       bar       foo       bar       foo       bar   
B                                                                         
A  1.074239 -2.461986  1.883436       NaN       NaN  0.094057 -0.356630   
B -2.285780  1.835541       NaN -1.958787 -0.629257       NaN -0.426053   
C  0.179040  1.325360  1.897414       NaN       NaN -0.858725  3.441882   

                                                    
A               three                two            
C       foo       bar      foo       bar       foo  
B                                                   
A  0.855697  1.771187      NaN       NaN -0.064347  
B  1.902528       NaN -0.64368 -2.309317       NaN  
C -0.779012 -2.756801      NaN       NaN -1.933443

El objeto resultante es un DataFrame con los índices jerárquicos en filas y columnas. Si no le pasamos el argumento 'values' entonces los datos podrían agregarse en los niveles jerárquicos superiores como por ejemplo:

In [104]:
pd.pivot_table(df, index=['B'], columns=['C'])

D                   E          
C       bar       foo       bar       foo
B                                        
A  0.068897 -0.586851 -1.088297 -0.996425
B -0.147507 -0.033373  0.054649 -0.133620
C  0.246889  0.586919  0.656783  0.229140

Para mejorar el output de una pivot table se puede usar el método .to_string() de la siguiente forma

In [50]:
table = pd.pivot_table(df, index=['A', 'B'], columns=['C'])

In [51]:
print(table.to_string(na_rep=''))

                D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A  0.537120 -1.230993 -0.178315  0.427849
      B -1.142890  0.917770 -0.213027  0.951264
      C  0.089520  0.662680  1.720941 -0.389506
three A  0.941718            0.885594          
      B           -0.979393           -0.321840
      C  0.948707           -1.378401          
two   A            0.047028           -0.032174
      B -0.314629           -1.154659          
      C           -0.429362           -0.966722


Si quieres la agregación de filas y columnas en los márgenes de la tabla puedes adicionarlos sin problema usando el argumento 'margins=True' de la siguiente forma:

In [52]:
df.pivot_table(index=['A', 'B'], columns='C', margins=True, aggfunc=np.std)

D                             E                    
C             bar       foo       All       bar       foo       All
A     B                                                            
one   A  0.441199  0.559776  1.100640  0.747674  1.939123  1.249886
      B  1.016677  1.295361  1.522926  0.073791  1.582687  1.135182
      C  1.019803  1.523745  1.109100  0.025774  0.283570  1.229507
three A  0.897218       NaN  0.897218  1.618485       NaN  1.618485
      B       NaN  1.804634  1.804634       NaN  0.357359  0.357359
      C  0.650295       NaN  0.650295  0.604312       NaN  0.604312
two   A       NaN  1.013625  1.013625       NaN  1.423220  1.423220
      B  1.965972       NaN  1.965972  1.164925       NaN  1.164925
      C       NaN  0.970980  0.970980       NaN  0.567152  0.567152
All      1.126707  1.246217  1.150420  1.313767  1.101303  1.160593

In [105]:
df.pivot_table?